## Jupyter notebook illustrating routines for postprocessing MLE segments 

Copyright 2019 Ursa Analytics, Inc.
Licensed under the Apache License, Version 2.0 (the "License");
You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

In [44]:
%matplotlib inline 
import matplotlib.font_manager as font_manager
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('/src/notebooks/')
import myIO,os,h5py
import scipy.linalg as sla 
import myIPyNBplt #collection of wrapper routines for custom plots and manipulation (sans some clutter coming with the flexibility of matplotlib)
import h5py,pandas

def loadh5(floc,dsetnameIndex=0,printFile=False,printDsets=False): #load a numeric hdf5 file. "floc" is the path of ASCII file to read.
    if floc[0]=='~':  #replace standard unix home shortcut with explicit path
        floc='/Users/calderoc' + floc[1:]
    f=h5py.File(floc,'r')
    dsetNames = f.keys()
    if printFile:
        print 'Read file: ' + floc
        print 'Pulling Dataset index', dsetnameIndex, 'Named: ' + dsetNames[dsetnameIndex]
    if printDsets:
        if printFile:
            print 'Displaying top level dsets in hdf5 file since printDset set to True'
            print 'Found N= ',len(f.keys()),'top level dsets (returning integer vs. any data since printDset=True)'
            for i in f.keys():
                print i
        return len(f.keys())
    else:
        dsetname=dsetNames[dsetnameIndex]
        g= f[dsetname]  #default name for h5import dataset when no arg given (so i use this in my scripts as well)
        g= np.array(g) #return numpy array (get shape with g.shape)
        f.close()
    return g 



In [80]:
import pandas
folder='./output/'
fname='cp2019_401'
suffix='_KF2DRes'
ftype='.h5'

Ntraj=401
dt=10/1000.
scalefac=.16
scalefac=1.
evDhist=[]
evKhist=[]
Labelhist=[]
valsH=[]

for graph in range(1,(Ntraj+1)):
# for graph in range(2,(5+1)):

    key=str(graph)+'_'
    
    
    with h5py.File(folder+os.sep+fname+ftype) as fsrc:
        print('-'*20)
        print(key)
        fsrcP = h5py.File(folder+os.sep+fname+suffix +ftype,'r')
        
        keys = [i for i in fsrc.keys() if key == i[0:len(key)]] #this contruct assumes that there are less than 10 "long" segments in a given trajectory.  if double digit long trajectories occur, need to modify
        dnet=[]
        dlist=[]

        for k in keys:
            vals=[]
            vals.append(k)
        
            print('reading key: ',k)
#             fig,ax,pltPars = myIPyNBplt.basicPltSetup(axisfontsize='22') #standard plot. set default in myIPyNBplt (helvetica recommended)
#             fig,ax,axTop,pltPars = myIPyNBplt.sharedXPltSetup(axisfontsize='22')
            d=fsrc[k].value
            x=d[:,2]
            vals.append(len(x))
            y=d[:,3]
            mx=np.mean(x)
            my=np.mean(y)
            dlist.append(np.array(d))
            dnet.extend(np.array(d))
            

            
            
            d=fsrcP[k].value #now plot eigenvalues of B and/or D
            pars=d[0]
            B=np.reshape(pars[0:4],(2,2))
            ev,evec = sla.eig(B)
            np.sort(np.real(ev))
            
            Sig = np.array([np.array([pars[4],0]),np.array([pars[5],pars[6]])])
            Dloc=np.dot(Sig,Sig.T)
            ev2,evec2 = sla.eig(Dloc)
            
            ratio=np.max(np.abs(ev2))/np.max(np.abs(ev))
            vals.append(ratio)
            vals.append(np.real(ev))
            vals.append(np.real(ev2))
            vals.append(np.real(evec))
            vals.append(np.real(evec2))
            
            
            v=np.vstack(([mx,my],[mx+evec[0,0],my+evec[1,0]])) #stack in order (B ev1,B ev2, then D ev1, D ev2)
            vals.append(np.real(v)) #
            v=np.vstack(([mx,my],[mx+evec[0,1],my+evec[1,1]])) #stack in order (B ev1,B ev2, then D ev1, D ev2)
            vals.append(np.real(v)) #
            v2=np.vstack(([mx,my],[mx+evec2[0,0],my+evec2[1,0]]))      
            vals.append(np.real(v2)) #
            v2=np.vstack(([mx,my],[mx+evec2[0,1],my+evec2[1,1]]))      
            vals.append(np.real(v2)) #
            
            valsH.extend([vals]) #store net list

                

            
            
        fsrcP.close()  

--------------------
1_
('reading key: ', u'1_1')
('reading key: ', u'1_2')
--------------------
2_
('reading key: ', u'2_1')
('reading key: ', u'2_2')
--------------------
3_
('reading key: ', u'3_1')
--------------------
4_
('reading key: ', u'4_1')
('reading key: ', u'4_2')
('reading key: ', u'4_3')
--------------------
5_
('reading key: ', u'5_1')
('reading key: ', u'5_2')
('reading key: ', u'5_3')
('reading key: ', u'5_4')
--------------------
6_
('reading key: ', u'6_1')
('reading key: ', u'6_2')
--------------------
7_
('reading key: ', u'7_1')
('reading key: ', u'7_2')
--------------------
8_
('reading key: ', u'8_1')
('reading key: ', u'8_2')
('reading key: ', u'8_3')
('reading key: ', u'8_4')
('reading key: ', u'8_5')
('reading key: ', u'8_6')
--------------------
9_
('reading key: ', u'9_1')
('reading key: ', u'9_2')
('reading key: ', u'9_3')
--------------------
10_
('reading key: ', u'10_1')
--------------------
11_
('reading key: ', u'11_1')
('reading key: ', u'11_2')
('r

In [81]:
valsH
df = pandas.DataFrame(valsH, columns = ['SegID', 'N','max(Diff)/max(Force)','Force Evals','Diff Evals','Force Evec','Diff Evec','Force Evec 1plt','Force Evec 2plt','Diff Evec 1plt','Diff Evec 2plt'])
df.to_csv('401_SegmentDetails.csv',index=False)
